In [1]:
import sys, logging
from pathlib import Path
import pandas as pd

In [2]:
# Define the project root and add it to the path so we can run HECTRE.
PROJ_ROOT = Path('..', '..').resolve()
sys.path.insert(0, str(PROJ_ROOT))
print(sys.path)

['/Users/jan535/code/HECTRE', '/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python311.zip', '/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11', '/opt/homebrew/Cellar/python@3.11/3.11.5/Frameworks/Python.framework/Versions/3.11/lib/python3.11/lib-dynload', '', '/Users/jan535/code/HECTRE/.venv/lib/python3.11/site-packages']


In [3]:
# Reload the notebook if imported modules are modified.
%load_ext autoreload
%autoreload 2

In [4]:
# Import HECTRE and disable logging. We'll be creating a custom report in this notebook.
from hectre.metrics import accuracy
logging.disable()

In [5]:
TESTING=True
TEST_ROOT = Path(PROJ_ROOT, 'hectre', 'tests', 'test_data')
control_cdfs = Path(TEST_ROOT, 'cdfs', 'eval_report').iterdir()
eval_data = {}
for control_cdf_path in control_cdfs:
    slug = control_cdf_path.stem.lower()
    pdf_path = Path(TEST_ROOT, slug + '.pdfdata')
    pdf_path_input = str(pdf_path)
    picos_input = "EASI-50;EASI-75;EASI-90"
    cdf_path_input = str(control_cdf_path)
    control_cdf = pd.read_csv(cdf_path_input)
    if TESTING:
        test_cdf = pd.read_csv(Path(TEST_ROOT, 'cdfs', 'eval_report_test_cdfs', control_cdf_path.name).resolve())
    else:
        test_cdf = accuracy.accuracy_extract_test(pdf_path_input, picos_input)
    acc = accuracy.cdf_accuracy(
        test_cdf,
        control_cdf,
        slug
    )
    eval_data[slug] = acc

set()
set()


In [6]:
print(eval_data.keys())

dict_keys(['287_silverberg_2021', '286_wollenberg_2021'])


In [7]:
wol = eval_data['286_wollenberg_2021']

In [8]:
print(wol.keys())

dict_keys(['comparison', 'lit_acc_pct', 'num_control_rows', 'matched_control_rows', 'clin_data_col_accs'])


# HECTRE Performance Evaluation Report

In [9]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
for slug, extraction in eval_data.items():
    print(slug)
    print(f"Literature data accuracy: {extraction['lit_acc_pct']:.2f}%")
    print("STACKED DF")
    display(extraction['comparison']['stacked_df'])
    print("TEST SIMILARITY TO CONTROL (%)")
    display(extraction['comparison']['comp_values_clin'])

287_silverberg_2021
Literature data accuracy: 85.07%
STACKED DF


ARM.BLIND  \
ARM.TRT      ARM.DOSE ARM.REGIMEN ENDPOINT ARM.TIME1 primary_key match_key sample                     
tralokinumab 600      q2w         easi     0         9           c9t9      similarity           100   
                                                                           control     Double-Blind   
             300.0    every2weeks easi75   16        9           c9t9      test        Double-Blind   
                                  easi90   16        7           c8t7      test        Double-Blind   
placebo      0        q2w         easi     16        8           c8t7      similarity           100   
                                                                           control     Double-Blind   
tralokinumab 300.0    every2weeks easi50   16        6           c7t6      test        Double-Blind   
placebo      0        q2w         easi     14        7           c7t6      similarity           100   
                                                                           control     Double-Blind   
tralokinumab 300.0    every2weeks easi75   16        5           c6t5      test        Double-Blind   
placebo      0        q2w         easi     12        6           c6t5      similarity           100   
                                                                           control     Double-Blind   
tralokinumab 300.0    every2weeks easi90   16        4           c5t4      test        Double-Blind   
placebo      0        q2w         easi     10        5           c5t4      similarity           100   
                                                                           control     Double-Blind   
tralokinumab 300.0    every4weeks easi75   32        2           c4t2      test        Double-Blind   
placebo      0        q2w         easi     8         4           c4t2      similarity           100   
                                                                           control     Double-Blind   
tralokinumab 300.0    every2weeks easi50   16        3           c3t3      test        Double-Blind   
placebo      0        q2w         easi     6         3           c3t3      similarity           100   
                                                                           control     Double-Blind   
tralokinumab 300.0    every2weeks easi75   32        8           c2t8      test        Double-Blind   
placebo      0        q2w         easi     4         2           c2t8      similarity           100   
                                                                           control     Double-Blind   
                                           2         1           c1t1      similarity           100   
                                                                           control     Double-Blind   
                      q2w         easi75   16        1           c1t1      test        Double-Blind   
             0        q2w         easi     0         0           c0t0      similarity           100   
                                                                           control     Double-Blind   
                      q2w         easi50   16        0           c0t0      test        Double-Blind   
tralokinumab 600      q2w         scorad   0         43                    similarity             0   
                                                                           control     Double-Blind   
             300      q2w         scorad   8         47                    similarity             0   
                                                                           control     Double-Blind   
                                           6         46                    similarity             0   
                                                                           control     Double-Blind   
                                           4         45                    similarity             0   
                                                                         

TEST SIMILARITY TO CONTROL (%)


ARM.BLIND  \
ARM.TRT      ARM.DOSE ARM.REGIMEN ENDPOINT ARM.TIME1 primary_key              
placebo      0        q2w         easi     0         0                  100   
                                           2         1                  100   
                                           4         2                  100   
                                           6         3                  100   
                                           8         4                  100   
                                           10        5                  100   
                                           12        6                  100   
                                           14        7                  100   
                                           16        8                  100   
tralokinumab 600      q2w         easi     0         9                  100   
             300      q2w         easi     2         10                   0   
                                           4         11                   0   
                                           6         12                   0   
                                           8         13                   0   
                                           10        14                   0   
                                           12        15                   0   
                                           14        16                   0   
                                           16        17                   0   
placebo      0        q2w         easi75   2         18                   0   
                                           4         19                   0   
                                           6         20                   0   
                                           8         21                   0   
                                           10        22                   0   
                                           12        23                   0   
                                           14        24                   0   
                                           16        25                   0   
tralokinumab 300      q2w         easi75   2         26                   0   
                                           4         27                   0   
                                           6         28                   0   
                                           8         29                   0   
                                           10        30                   0   
                                           12        31                   0   
                                           14        32                   0   
                                           16        33                   0   
placebo      0        q2w         scorad   0         34                   0   
                                           2         35                   0   
                                           4         36                   0   
                                           6         37                   0   
                                           8         38                   0   
                                           10        39                   0   
                                           12        40                   0   
                                           14        41                   0   
                                           16        42                   0   
tralokinumab 600      q2w         scorad   0         43                   0   
             300      q2w         scorad   2         44                   0   
                                           4         45                   0   
                                           6         46                   0   
                                           8         47                   0   
                                           10        48                   0   
                                     

286_wollenberg_2021
Literature data accuracy: 74.60%
STACKED DF


ARM.BLIND  \
ARM.TRT      ARM.DOSE ARM.REGIMEN ENDPOINT ARM.TIME1 primary_key match_key sample                     
tralokinumab 300.0    every2weeks easi75   16        11          c9t11     test        Double-Blind   
             300      q2w         easi75   4         9           c9t11     similarity           100   
                                                                           control     Double-Blind   
             300.0    every2weeks easi75   52        12          c8t12     test        Double-Blind   
             300      q2w         easi75   2         8           c8t12     similarity           100   
...                                                                                             ...   
placebo      0        q2w         easi     10        56                    control     Double-Blind   
                                           0         140                   similarity             0   
                                                                           control     Double-Blind   
                                                     51                    similarity             0   
                                                                           control     Double-Blind   

                                                                                      ARM.RANDFLG  \
ARM.TRT      ARM.DOSE ARM.REGIMEN ENDPOINT ARM.TIME1 primary_key match_key sample                   
tralokinumab 300.0    every2weeks easi75   16        11          c9t11     test              True   
             300      q2w         easi75   4         9           c9t11     similarity         100   
                                                                           control           True   
             300.0    every2weeks easi75   52        12          c8t12     test              True   
             300      q2w         easi75   2         8           c8t12     similarity         100   
...                                                                                           ...   
placebo      0        q2w         easi     10        56                    control           True   
                                           0         140                   similarity           0   
                                                                           control           True   
                                                     51                    similarity           0   
                                                                           control           True   

                                                                                            ARM.TRT  \
ARM.TRT      ARM.DOSE ARM.REGIMEN ENDPOINT ARM.TIME1 primary_key match_key sample                     
tralokinumab 300.0    every2weeks easi75   16        11          c9t11     test        Tralokinumab   
             300      q2w         easi75   4         9           c9t11     similarity           100   
                                                                           control     tralokinumab   
             300.0    every2weeks easi75   52        12          c8t12     test        Tralokinumab   
             300      q2w         easi75   2         8           c8t12     similarity           100   
...                                                                                             ...   
placebo      0        q2w         easi     10        56                    control          placebo   
                                           0         140                   similarity             0   
                                                                           control          placebo   
                                                     51                    similarity             0   
                                                                           control          placebo   

                                                                                                

TEST SIMILARITY TO CONTROL (%)


ARM.BLIND  \
ARM.TRT      ARM.DOSE ARM.REGIMEN ENDPOINT ARM.TIME1 primary_key              
placebo      0        q2w         easi75   2         0                  100   
                                           4         1                  100   
                                           6         2                  100   
                                           8         3                  100   
                                           10        4                  100   
                                           12        5                  100   
                                           14        6                  100   
                                           16        7                  100   
tralokinumab 300      q2w         easi75   2         8                  100   
                                           4         9                  100   
                                           6         10                 100   
                                           8         11                 100   
                                           10        12                 100   
                                           12        13                 100   
                                           14        14                 100   
                                           16        15                   0   
                                           52        16                   0   
                      q4w         easi75   52        17                   0   
placebo      0        q2w         easi75   52        18                   0   
                                  easi50   2         19                   0   
                                           4         20                   0   
                                           6         21                   0   
                                           8         22                   0   
                                           10        23                   0   
                                           12        24                   0   
                                           14        25                   0   
                                           16        26                   0   
tralokinumab 300      q2w         easi50   2         27                   0   
                                           4         28                   0   
                                           6         29                   0   
                                           8         30                   0   
                                           10        31                   0   
                                           12        32                   0   
                                           14        33                   0   
                                           16        34                   0   
placebo      0        q2w         easi90   2         35                   0   
                                           4         36                   0   
                                           6         37                   0   
                                           8         38                   0   
                                           10        39                   0   
                                           12        40                   0   
                                           14        41                   0   
                                           16        42                   0   
tralokinumab 300      q2w         easi90   2         43                   0   
                                           4         44                   0   
                                           6         45                   0   
                                           8         46                   0   
                                           10        47                   0   
                                           12        48                   0   
                                     